In [1]:
import pandas as pd
import numpy as np

In [12]:
player_stats = pd.read_csv('player_stats.csv')

In [16]:
class OddsGenerator:
    
    def __init__(self, dataframe):
        self.df = dataframe
        self.stat_categories = ['PTS', 'REB', 'AST']
    
    def generate_all_lines(self):
        lines = []
        players = self.df['Player'].unique()
        
        for player in players:
            player_data = self.df[self.df['Player'] == player]
            
            for stat in self.stat_categories:
                stat_values = player_data[stat].dropna()
                
                if len(stat_values) > 0:
                    median_value = stat_values.median()
                    line = round(median_value * 2) / 2
                    
                    lines.append({
                        'Player': player,
                        'stat_category': stat,
                        'line': line,
                        'games_sample_size': len(stat_values)
                    })
        
        return pd.DataFrame(lines)
    
    def generate_lines_with_validation(self):
        lines = []
        players = self.df['Player'].unique()
        
        for player in players:
            player_data = self.df[self.df['Player'] == player]
            
            for stat in self.stat_categories:
                stat_values = player_data[stat].dropna()
                
                if len(stat_values) > 0:
                    median_value = stat_values.median()
                    line = round(median_value * 2) / 2
                    
                    over_count = (stat_values > line).sum()
                    under_count = (stat_values < line).sum()
                    push_count = (stat_values == line).sum()
                    
                    total = len(stat_values)
                    over_pct = (over_count / total) * 100
                    under_pct = (under_count / total) * 100
                    push_pct = (push_count / total) * 100
                    
                    lines.append({
                        'Player': player,
                        'stat_category': stat,
                        'line': line,
                        'mean': stat_values.mean(),
                        'median': median_value,
                        'std_dev': stat_values.std(),
                        'games_sample_size': total,
                        'over_pct': round(over_pct, 1),
                        'under_pct': round(under_pct, 1),
                        'push_pct': round(push_pct, 1)
                    })
        
        return pd.DataFrame(lines)
    
    def export_lines_simple(self, filename='betting_lines.csv'):
        lines_df = self.generate_all_lines()
        lines_df.to_csv(filename, index=False)
        print(f"Lines exported to {filename}")
        return lines_df
    
    def export_lines_detailed(self, filename='betting_lines_detailed.csv'):
        lines_df = self.generate_lines_with_validation()
        lines_df.to_csv(filename, index=False)
        print(f"Detailed lines exported to {filename}")
        return lines_df


if __name__ == "__main__":
    generator = OddsGenerator(player_stats)
    
    print("=== Simple Betting Lines ===")
    simple_lines = generator.generate_all_lines()
    print(simple_lines)
    print()
    
    print("=== Detailed Lines with Validation ===")
    detailed_lines = generator.generate_lines_with_validation()
    print(detailed_lines)
    
    generator.export_lines_simple('betting_lines.csv')
    generator.export_lines_detailed('betting_lines_detailed.csv')

=== Simple Betting Lines ===
           Player stat_category  line  games_sample_size
0     Luka Dončić           PTS  36.0                  1
1     Luka Dončić           REB  10.5                  1
2     Luka Dončić           AST   9.5                  1
3    Devin Booker           PTS  33.5                  1
4    Devin Booker           REB   4.0                  1
..            ...           ...   ...                ...
937  Jared McCain           REB   1.0                  1
938  Jared McCain           AST   2.0                  1
939  Jevon Carter           PTS   0.0                  1
940  Jevon Carter           REB   0.0                  1
941  Jevon Carter           AST   2.0                  1

[942 rows x 4 columns]

=== Detailed Lines with Validation ===
           Player stat_category  line  mean  median  std_dev  \
0     Luka Dončić           PTS  36.0  36.0    36.0      NaN   
1     Luka Dončić           REB  10.5  10.7    10.7      NaN   
2     Luka Dončić           AST